In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.6 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

NLP Deep Dive: RNNs

Text Preprocessing

Tokenization

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

'I caught the North American premiere of this at the Chicago International F'

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#104) ['I','caught','the','North','American','premiere','of','this','at','the','Chicago','International','Film','Festival','.','I','was','beyond','disappointed','.','From','the','mood','in','the','audience',',','I','was',"n't"...]


In [7]:
first(spacy(['The U.S. dollar 1is1.00.']))

(#5) ['The','U.S.','dollar','1is1.00','.']

Unable to replicate the font. Probably in Latex.

Word Tokenization with fastai

In [8]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#118) ['xxbos','i','caught','the','xxmaj','north','xxmaj','american','premiere','of','this','at','the','xxmaj','chicago','xxmaj','international','xxmaj','film','xxmaj','festival','.','i','was','beyond','disappointed','.','xxmaj','from','the','mood'...]


In [9]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [10]:
coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Subword Tokenization

In [11]:
txts = L(o.open().read() for o in files[:2000])

In [12]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [13]:
subword(1000)

'▁I ▁c aught ▁the ▁No r th ▁American ▁pre mi er e ▁of ▁this ▁at ▁the ▁Ch ic ag o ▁In ter n ation al ▁F il m ▁F est iv al . ▁I ▁was ▁be y on d ▁disappointed'

In [14]:
subword(200)

'▁I ▁ca u g h t ▁the ▁ N or th ▁A m er ic an ▁p re m i e re ▁of ▁this ▁a t ▁the ▁ C h ic a g o ▁I n ter n at i'

In [15]:
subword(10000)

"▁I ▁caught ▁the ▁North ▁American ▁premiere ▁of ▁this ▁at ▁the ▁Chicago ▁Inter national ▁Film ▁Festival . ▁I ▁was ▁beyond ▁disappointed . ▁Fro m ▁the ▁mood ▁in ▁the ▁audience , ▁I ▁wasn ' t ▁the ▁only ▁one . < br ▁/>"

Numericalization

In [16]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#118) ['xxbos','i','caught','the','xxmaj','north','xxmaj','american','premiere','of','this','at','the','xxmaj','chicago','xxmaj','international','xxmaj','film','xxmaj','festival','.','i','was','beyond','disappointed','.','xxmaj','from','the','mood'...]


In [17]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#118) ['xxbos','i','caught','the','xxmaj','north','xxmaj','american','premiere','of'...]

In [18]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#1992) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','to','of','is','i','it','in'...]"

In [19]:
nums = num(toks)[:20]; nums

TensorText([  2,  17, 826,   9,   8, 703,   8, 280,   0,  15,  21,  45,   9,   8,   0,   8, 974,   8,  35,   8])

In [20]:
' '.join(num.vocab[o] for o in nums)

'xxbos i caught the xxmaj north xxmaj american xxunk of this at the xxmaj xxunk xxmaj international xxmaj film xxmaj'

Putting Our Texts into Batches for a Language Model

In [21]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [22]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [23]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [24]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [25]:
nums200 = toks200.map(num)

In [26]:
dl = LMDataLoader(nums200)

In [27]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [34]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos i caught the xxmaj north xxmaj american xxunk of this at the xxmaj xxunk xxmaj international xxmaj film xxmaj'

In [33]:
' '.join(num.vocab[o] for o in y[0][:20])

'i caught the xxmaj north xxmaj american xxunk of this at the xxmaj xxunk xxmaj international xxmaj film xxmaj festival'

Training a Text Classifier